In [ ]:
# api
import json
import requests

# sunrise and sunset
import datetime as dt
from astral.sun import sun
from astral import LocationInfo

# long and lat
from geopy.geocoders import Nominatim

# utils
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# prefs
plt.style.use('fivethirtyeight')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

In [ ]:
# api
api_get = 'C03002_003E,C03002_004E,C03002_005E,C03002_006E,C03002_007E,C03002_008E,C03002_009E,C03002_012E'
api_for = 'county:*'
with open('.census_api_key') as f:
    api_key = json.load(f)['api_key']
    
# construct the api call we will use
api_url = f'https://api.census.gov/data/2017/acs/acs1?get={api_get}&for={api_for}&key={api_key}'

# call the api and collect the response
response = requests.get(api_url)

cols = ['White',
        'Black',
        'Indigenous',
        'Asian',
        'Pacific_Islander',
        'Other',
        'Mixed',
        'Hispanic',
        'State',
        'County']

# philidelphia city and county are coextensive
census = pd.DataFrame(json.loads(response.text)[1:], columns=cols)
philly = census[(census.State=='42')&(census.County=='101')].astype('int').reset_index(drop=True)

# merge cols
philly.Other += philly.Indigenous + philly.Mixed
philly.Asian += philly.Pacific_Islander
philly = philly.drop(columns=['Indigenous', 'Pacific_Islander', 'Mixed', 'State', 'County'])
philly_normed = round(philly/philly.iloc[0].sum(), 3)

# display pop. by race
print(f"{philly.columns[0]}\t{philly.columns[1]}\t{philly.columns[2]}\t{philly.columns[3]}\t{philly.columns[4]}")
print(f"{philly.values[0][0]}\t{philly.values[0][1]}\t{philly.values[0][2]}\t{philly.values[0][3]}\t{philly.values[0][4]}")
print(f"{philly_normed.values[0][0]}\t{philly_normed.values[0][1]}\t{philly_normed.values[0][2]}\t{philly_normed.values[0][3]}\t{philly_normed.values[0][4]}")
# print(pd.concat([philly, philly_normed]).reset_index().T.rename(columns={0:'Total Pop.', 1:'Percentage'}).T.drop(columns=['index'])) #<-- alt. way to display pop. table

In [ ]:
# philadelphia lat and long centroid
address='Philadelphia'
with open('.geopy_user_agent') as f:
    geopy_user_agent = json.load(f)['geopy_user_agent']

geolocator = Nominatim(user_agent=geopy_user_agent)
location = geolocator.geocode(address)
print((location.latitude, location.longitude))

In [ ]:
# read in police stops dataset
stops = pd.read_csv('data/pa_philadelphia_2020_04_01.csv')

display(stops.head(3))
stops.info()

In [ ]:
# merge other and unknown subject race
stops.subject_race = np.where((stops.subject_race == 'other') | (stops.subject_race == 'unknown'),
                              'other/unknown',
                              stops.subject_race)

# rename columns to match census format
conds = [stops.subject_race == 'white',
         stops.subject_race == 'black',
         stops.subject_race == 'asian/pacific islander',
         stops.subject_race == 'hispanic',
         stops.subject_race == 'other/unknown']

stops.subject_race = np.select(conds, ['White',
                                       'Black',
                                       'Asian',
                                       'Hispanic',
                                       'Other'])

In [ ]:
# convert 'date' to date_time obj
stops.date = pd.to_datetime(stops.date)

# display range of dates (start to end)
print(min(stops.date).date())
print(max(stops.date).date())
# stops.date.agg(['min', 'max']) #<-- alt. way to display date range

In [ ]:
# drop 2018 (partial year)
stops = stops[stops.date < '2018-01-01']

# display new end date and new number of stops
print(max(stops.date).date())
display(len(stops))

In [ ]:
# display types of stops
print(stops.type.value_counts())

In [ ]:
# drop pedestrian stops
stops = stops[stops.type == 'vehicular']

# check types of stops and display new number of stops
print(stops.type.value_counts())
display(len(stops))

In [ ]:
# display stops by year
display("Stops/Year",
        "n",
        stops.date.map(lambda x: x.year).value_counts(),
        "prop",
        stops.date.map(lambda x: x.year).value_counts(normalize=True))

# display stops by race
display("Stops/Race",
        "n",
        stops.subject_race.value_counts(),
        "prop",
        stops.subject_race.value_counts(normalize=True))

How to explain racial disparity?

In [ ]:
# check stops per year per race
stops.groupby([stops.date.map(lambda x: x.year), stops.subject_race]).size()

This table has too many entries to digest quickly. Visualizing the stops per year per race will be more effective.

In [ ]:
# visualize trend
fig = plt.figure(figsize=(7,7))
ax = plt.axes()
sns.lineplot(data=stops.groupby([stops.date.map(lambda x: x.year),
                                 stops.subject_race]).size().unstack(level=1),
             dashes=False,
             markers=["o"]*5,
             linewidth=1.5)
# plt.setp(ax.lines, linewidth=2)
ax.set_xticks([2014, 2015, 2016, 2017])
ax.set_ylabel('n', )
plt.legend(loc='center right', title='subject_race', bbox_to_anchor=(1.25, .75))
plt.show()

Black subject stops continue increasing past 2015 while other races plateau. Data should be analyzed annualy. This trend dissapears if pedestrian stops included

...

looking at trends by sub-categories can often be very helpful. (E.g., in Nashville, looking at the different listed stop reasons uncovers the extent of the disparities.)

Examine year 2017 -- Benchmark tests: establish baseline (are blacks stopped more because they make up more of the population?)

In [ ]:
# get stops for year 2017 only
stops_2017 = stops[stops.date.map(lambda x: x.year) == 2017]

# get stops per race for year 2017 only
race_2017 = pd.concat([stops_2017.groupby(stops.subject_race).size(),
                philly.T], axis=1, sort=True)
race_2017.columns = ['n', 'num_people']

# get 2017 stop rate
race_2017['stop_rate'] = race_2017.n / race_2017.num_people

display(race_2017)

In [ ]:
# black and white stop rates
display(race_2017.stop_rate['Black'] / race_2017.stop_rate['White'])

# black and hispanic stop rates
display(race_2017.stop_rate['Hispanic'] / race_2017.stop_rate['White'])

In [ ]:
# check frisk/search values
display(stops_2017.search_conducted.value_counts())
display(stops_2017.frisk_performed.value_counts())

In [ ]:
# get search and frisk rates per race
search_frisk = pd.concat([stops_2017.groupby('subject_race').search_conducted.mean(),
                          stops_2017.groupby('subject_race').frisk_performed.mean()], axis=1)
search_frisk.columns = ['search_rate', 'frisk_rate']

display(search_frisk)

In [ ]:
# black and white search rates
display(search_frisk.search_rate['Black'] / search_frisk.search_rate['White'])

# black and white frisk rates
display(search_frisk.frisk_rate['Black'] / search_frisk.frisk_rate['White'])

# hispanic and white stfriskop rates
display(search_frisk.search_rate['Hispanic'] / search_frisk.search_rate['White'])

# hispanic and white frisk rates
display(search_frisk.frisk_rate['Hispanic'] / search_frisk.frisk_rate['White'])

Problems with out benchmark test:

1. Our census pop. data doesn't capture what the distribution of driving behavior looks like (Philly residents != Philly drivers)

2. Rates of justifiable searches might also vary by race, so we need to check outcomes of searches (hit_rate)

In [ ]:
stops_2017.contraband_found.value_counts()

In [ ]:
stops.isna().sum()

In [ ]:
stops[stops.search_conducted==True].groupby(['subject_race', 'contraband_found']).size()

In [ ]:
searches_2017 = stops_2017[stops_2017.search_conducted==True]
searches_2017.contraband_found = searches_2017.contraband_found.astype(bool)
dff = pd.DataFrame(searches_2017.groupby('subject_race').contraband_found.mean()).rename(columns={'contraband_found':'hit_rate'})
display(dff)

Are officers searching non-white drivers based on less evidence? Do hit rates vary by precint?

In [ ]:
# examine hit rate by district
pd.DataFrame(searches_2017.groupby([searches_2017.subject_race, searches_2017.district]).contraband_found.mean()).rename(columns={'contraband_found':'hit_rate'})

In [ ]:
pd.Index(list(range(1, 23))*2)

In [ ]:
ddff = pd.DataFrame(searches_2017.groupby([searches_2017.subject_race, searches_2017.district]).contraband_found.mean()).rename(columns={'contraband_found':'hit_rate'}).unstack().T.melt(id_vars='White')
ddff = ddff[ddff.subject_race.isin(['Black', 'Hispanic'])]
ddff.fillna(0)
ddff = ddff.set_index(pd.Index(np.tile(stops.district.unique(), 2)))
ddff = ddff.rename(columns={'White':'white_hit_rate', 'value':'minority_hit_rate'})
ddff.sort_index()

Exclude district 77 (airport, da offices, etc.)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(12,6))
sns.scatterplot(x='white_hit_rate',
                y='minority_hit_rate',
                size=stops_2017.groupby('district').size(),
                data=ddff[ddff.subject_race=='Black'],
                edgecolor='black',
                color="white",
                sizes=(1, 400),                ax=ax[0]).set_title('Black')
ax[0].plot([-0.05, .85], [-0.05, .85], linewidth=0.5, linestyle='-.', color='k')

sns.scatterplot(x='white_hit_rate',
                y='minority_hit_rate',
                size=stops_2017.groupby('district').size(),
                data=ddff[ddff.subject_race=='Hispanic'],
                edgecolor='black',
                color="white",
                sizes=(1, 400),
                ax=ax[1]).set_title('Hispanic')
ax[1].plot([-0.05, .85], [-0.05, .85], linewidth=0.5, linestyle='-.', color='k')
plt.setp(ax, xlim=(-0.05, .85), ylim=(-0.05, .85))
ax[0].get_legend().remove()
ax[1].get_legend().remove()
plt.show()

In [ ]:
stops_2017[stops_2017.district == 77].location.value_counts()

In [ ]:
# compute citywide hit rates again with district 77 removed
searches_2017 = stops_2017[(stops_2017.search_conducted==True) & (stops_2017.district!=77)]
searches_2017.contraband_found = searches_2017.contraband_found.astype(bool)
dff = pd.DataFrame(searches_2017.groupby('subject_race').contraband_found.mean()).rename(columns={'contraband_found':'hit_rate'})
display(dff)

Veil of Darkness Test

In [ ]:
# set city location for sunrise and sunset
city = LocationInfo("Philadelphia",
                    "Pennsylvania",
                    "Eastern Standard Time", #<--- check which params are req'd
                    location.latitude,
                    location.longitude)

# convert time to datetime.time obj
stops_2017['time']    = stops_2017.time.apply(lambda x: pd.to_datetime(x).time())
                     
# add sunset and dusk
stops_2017['Sunset']  = stops_2017.date.apply(lambda x: sun(city.observer, x, tzinfo='EST')["sunset"].time())
stops_2017['Dusk']    = stops_2017.date.apply(lambda x: sun(city.observer, x, tzinfo='EST')["dusk"].time())

# calculate whether or not stop is after dusk
stops_2017['is_dark'] = stops_2017.time > stops_2017.Dusk

# drop stops is subject is neither black nor white
stops_2017 = stops_2017.loc[(stops_2017.subject_race=='Black') | (stops_2017.subject_race=='White')]

# drop stops if stop time is before/after annual sunset.min()/dusk.max()
stops_2017 = stops_2017.loc[(stops_2017.time > stops_2017.Sunset.min()) & (stops_2017.time < stops_2017.Dusk.max())]

# drop stops if stop is after sunset for that date but before dusk for that date
stops_2017 = stops_2017.loc[(stops_2017.time < stops_2017.Sunset) | (stops_2017.time > stops_2017.Dusk)]

In [ ]:
aaa = stops_2017.loc[(stops_2017.time >= pd.to_datetime('17:30:00').time()) & (stops_2017.time < pd.to_datetime('18:00:00').time())]
bbb = stops_2017.loc[(stops_2017.time >= pd.to_datetime('18:00:00').time()) & (stops_2017.time < pd.to_datetime('18:30:00').time())]
ccc = stops_2017.loc[(stops_2017.time >= pd.to_datetime('18:30:00').time()) & (stops_2017.time < pd.to_datetime('19:00:00').time())]
ddd = stops_2017.loc[(stops_2017.time >= pd.to_datetime('19:00:00').time()) & (stops_2017.time < pd.to_datetime('19:30:00').time())]
eee = stops_2017.loc[(stops_2017.time >= pd.to_datetime('19:30:00').time()) & (stops_2017.time < pd.to_datetime('20:00:00').time())]
fff = stops_2017.loc[(stops_2017.time >= pd.to_datetime('20:00:00').time()) & (stops_2017.time < pd.to_datetime('20:30:00').time())]

In [ ]:
aaa['is_black'] = aaa.subject_race=='Black'
bbb['is_black'] = bbb.subject_race=='Black'
ccc['is_black'] = ccc.subject_race=='Black'
ddd['is_black'] = ddd.subject_race=='Black'
eee['is_black'] = eee.subject_race=='Black'
fff['is_black'] = fff.subject_race=='Black'

In [ ]:
print(
    aaa.groupby('is_dark').is_black.mean(),
    bbb.groupby('is_dark').is_black.mean(),
    ccc.groupby('is_dark').is_black.mean(),
    ddd.groupby('is_dark').is_black.mean(),
    eee.groupby('is_dark').is_black.mean()
)